In [1]:
import yfinance as yf

ticker = yf.Ticker('TSLA')
df = ticker.history(period='5d')

In [2]:
display(df)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-10-14 00:00:00-04:00,220.130005,221.910004,213.740005,219.160004,86291900,0.0,0.0
2024-10-15 00:00:00-04:00,220.009995,224.259995,217.119995,219.570007,62988800,0.0,0.0
2024-10-16 00:00:00-04:00,221.399994,222.820007,218.929993,221.330002,49632800,0.0,0.0
2024-10-17 00:00:00-04:00,221.589996,222.080002,217.899994,220.889999,50791800,0.0,0.0
2024-10-18 00:00:00-04:00,220.710007,222.279999,219.229996,220.699997,49526800,0.0,0.0


In [3]:
display(df['Close'])

Date
2024-10-14 00:00:00-04:00    219.160004
2024-10-15 00:00:00-04:00    219.570007
2024-10-16 00:00:00-04:00    221.330002
2024-10-17 00:00:00-04:00    220.889999
2024-10-18 00:00:00-04:00    220.699997
Name: Close, dtype: float64

Shifts the rows by 2 so we can have a comparison on "Close" values for each row in a window of 2 days.

This is achieved by creating a new DataFrame where each row has an extra column `2DaysShift` which respresents the `Close` value 2 days before.

This is possible with this dataset given that each row represents a day and theres no missing row values.

In [5]:
import pandas as pd

display(pd.concat([df['Close'], df['Close'].shift(2)], axis=1, keys=['Close', '2DaysShift']))

,Close,2DaysShift
Date,,
2024-10-14 00:00:00-04:00,219.160004,NaN
2024-10-15 00:00:00-04:00,219.570007,NaN
2024-10-16 00:00:00-04:00,221.330002,219.160004
2024-10-17 00:00:00-04:00,220.889999,219.570007
2024-10-18 00:00:00-04:00,220.699997,221.330002


Get the % difference between the `2DaysShift` and `Close`

$$\text{PC} = \frac{A - B}{B}$$

Where:

**A**: Current Price
**B**: Earlier Price (`2DaysShift`)
**PC**: Percentage Change

In [8]:
(df['Close'] - df['Close'].shift(2)) / df['Close'].shift(2)

Date
2024-10-14 00:00:00-04:00         NaN
2024-10-15 00:00:00-04:00         NaN
2024-10-16 00:00:00-04:00    0.009901
2024-10-17 00:00:00-04:00    0.006012
2024-10-18 00:00:00-04:00   -0.002846
Name: Close, dtype: float64

For more precision in financial analysis context a different formula is recommended:

$$y = \ln\left(\frac{A}{B}\right)$$

In [9]:
import numpy as np

df['2daysRise'] = np.log(df['Close'] / df['Close'].shift(2))
display(df)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,2daysRise
Date,,,,,,,,
2024-10-14 00:00:00-04:00,220.130005,221.910004,213.740005,219.160004,86291900,0.0,0.0,NaN
2024-10-15 00:00:00-04:00,220.009995,224.259995,217.119995,219.570007,62988800,0.0,0.0,NaN
2024-10-16 00:00:00-04:00,221.399994,222.820007,218.929993,221.330002,49632800,0.0,0.0,0.009853
2024-10-17 00:00:00-04:00,221.589996,222.080002,217.899994,220.889999,50791800,0.0,0.0,0.005994
2024-10-18 00:00:00-04:00,220.710007,222.279999,219.229996,220.699997,49526800,0.0,0.0,-0.002851


Rolling Window Calculation is also common, which consists on comparing each value with the average value over _n_ periods.

> Every pandas object has a rolling() method for looking at a rolling win- dow of values.

In [11]:
df['2daysAvg'] = df['Close'].shift(1).rolling(2).mean()
display(df[['Close', '2daysAvg']])

,Close,2daysAvg
Date,,
2024-10-14 00:00:00-04:00,219.160004,NaN
2024-10-15 00:00:00-04:00,219.570007,NaN
2024-10-16 00:00:00-04:00,221.330002,219.365005
2024-10-17 00:00:00-04:00,220.889999,220.450005
2024-10-18 00:00:00-04:00,220.699997,221.110001


- Use `shift(1)` to exclude the current day data and use prev day instead when calculating the average.
- Use `rolling(2)` to indicate that we want to draw on two consecutive rows.
- Use `mean()` to calculate average for each pair of consecutive rows.

Now the Percentage Change between each day's price can be calculated with its associated rolling average.

In [12]:
df['2daysAvgRise'] = np.log(df['Close'] / df['2daysAvg'])
display(df[['Close','2daysRise','2daysAvgRise']])

,Close,2daysRise,2daysAvgRise
Date,,,
2024-10-14 00:00:00-04:00,219.160004,NaN,NaN
2024-10-15 00:00:00-04:00,219.570007,NaN,NaN
2024-10-16 00:00:00-04:00,221.330002,0.009853,0.008918
2024-10-17 00:00:00-04:00,220.889999,0.005994,0.001994
2024-10-18 00:00:00-04:00,220.699997,-0.002851,-0.001856
